In [3]:
import os
import sys, traceback
import argparse
import csv
from datetime import datetime
from functools import reduce
import glob
from itertools import groupby
import numpy as np
import pandas as pd
from functools import partial
from multiprocessing import Pool
from pybedtools import BedTool
import subprocess



In [4]:
core_breaksFs = glob.glob("/dors/capra_lab/projects/enhancer_ages/fantom/data/shuffle/breaks/*age_breaks.bed")

test_path = "/dors/capra_lab/projects/enhancer_ages/fantom/data/download"

In [5]:
core_breaksFs

['/dors/capra_lab/projects/enhancer_ages/fantom/data/shuffle/breaks/shuf-all_fantom_enh_112_tissues-54_age_breaks.bed',
 '/dors/capra_lab/projects/enhancer_ages/fantom/data/shuffle/breaks/shuf-all_fantom_enh_112_tissues-64_age_breaks.bed',
 '/dors/capra_lab/projects/enhancer_ages/fantom/data/shuffle/breaks/shuf-all_fantom_enh_112_tissues-49_age_breaks.bed',
 '/dors/capra_lab/projects/enhancer_ages/fantom/data/shuffle/breaks/shuf-all_fantom_enh_112_tissues-17_age_breaks.bed',
 '/dors/capra_lab/projects/enhancer_ages/fantom/data/shuffle/breaks/shuf-all_fantom_enh_112_tissues-14_age_breaks.bed',
 '/dors/capra_lab/projects/enhancer_ages/fantom/data/shuffle/breaks/shuf-all_fantom_enh_112_tissues-89_age_breaks.bed',
 '/dors/capra_lab/projects/enhancer_ages/fantom/data/shuffle/breaks/shuf-all_fantom_enh_112_tissues-15_age_breaks.bed',
 '/dors/capra_lab/projects/enhancer_ages/fantom/data/shuffle/breaks/shuf-all_fantom_enh_112_tissues-37_age_breaks.bed',
 '/dors/capra_lab/projects/enhancer_ages

In [6]:

def loadConstants(species):  # note chrom.sizes not used in current implementation | 2018.10.29
    return {'hg19': ("/dors/capra_lab/users/bentonml/data/dna/hg19/hg19_blacklist_gap.bed", "/dors/capra_lab/data/dna/human/hg19/hg19_trim.chrom.sizes"),
            'hg38': ("/dorvs/capra_lab/users/bentonml/data/dna/hg38/hg38_blacklist_gap.bed", "/dors/capra_lab/data/dna/human/hg38/hg38_trim.chrom.sizes"),
            'mm10': ("/dors/capra_lab/users/bentonml/data/dna/mm10/mm10_blacklist_gap.bed", "/dors/capra_lab/data/dna/mouse/mm10/mm10_trim.chrom.sizes")
            }[species]

def mkdir(path):
    if os.path.isdir(path) == False:
        cmd = "mkdir %s" % path
        os.system(cmd)
        
def os_remove(files):
    cmd = "rm %s" % files
    os.system(cmd)

def stripTabs(infile, tempfile):
    cmd = '''tr -s " \t" < %s > %s && mv %s %s''' % (infile, tempfile, tempfile, infile)
    os.system(cmd)
    
    rm_cmd = "rm %s" % tempfile
    os.system(cmd)
    

In [14]:
for core_breaks_file in core_breaksFs:
    sample_id = (core_breaks_file.split("/")[-1]).split(".")[0]
    inpath = "%s/breaks/" % test_path 
    mkdir(inpath)

    outpath = "%s/tfbs/"% test_path
    mkdir(outpath)
    #####
    # prepare workspace
    #####

    outfile_wao = "%s%s_x_raw_tfbs_midpeak.bed"% (outpath, sample_id)
    outfile_waoTemp = "%stemp.bed"% (outpath)

    enh_out = "%s%s_enh_tfbs_density.bed" % (outpath, sample_id)
    syn_out = "%s%s_syn_tfbs_density.bed" % (outpath, sample_id)
    print(syn_out)
    tfbs_file = "/dors/capra_lab/data/encode/midpeaks_wgEncodeRegTfbsClusteredV3.bed" # TFBS data, 30bp trimmed ChIP-Peaks

    mkdir(outpath) # make directory

    ########
    # PART 1 - sort bed file
    ########
    c = BedTool(core_breaks_file).sort()

    ########
    # PART 2 - Bed intersect file with TFBS
    ########
    t = BedTool(tfbs_file).sort()
    c.intersect(t, wao =True).saveas(outfile_wao)

    stripTabs(outfile_wao, outfile_waoTemp)

    ########
    # open the sample_id file
    ########

    enh = pd.read_csv(outfile_wao, header= None, sep= '\t',low_memory = False)
    print(len(enh), "raw df len")

    ########
    # reformat file to get columns/data you want
    ########

    print("the number syntenic blocks that did not overlap TFBS" ,\
          len(enh.iloc[:,-1].loc[enh[20]==0]))

    #select the columns you want for analyzing break density
    enh = enh[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 20]].drop_duplicates()

    # rename the columns
    enh.columns = ["chr_syn", "start_syn", "end_syn", "enh_id", 
                   "chr_enh", "start_enh", "end_enh","seg_index", 
                   "core_remodeling", "core", "mrca","chr_tf", 
                   "start_tf", "end_tf", "tf", "len_tfbs_overlap"]

    # assign unique tf_id

    enh["tf_id"] = enh["chr_tf"] + ":" + enh["start_tf"].map(str) + "-" + enh["end_tf"].map(str) +"_"+ enh["tf"]

    print("the number of unique enhancers", len(enh["enh_id"].unique()))

    enh_tf_list = enh["enh_id"].unique() # make a list of unique enhancer ids

    enh_tf_list = enh_tf_list
    val = 0

    # int transform
    enh[["start_enh", "end_enh", "start_tf", "start_syn","end_syn","end_tf", "seg_index", "core_remodeling",       "core","len_tfbs_overlap"]] = enh[["start_enh",  "end_enh", "start_tf","start_syn", "end_syn", 
         "end_tf", "seg_index", "core_remodeling", "core","len_tfbs_overlap"]].astype(int)

    enh["mrca"] = enh["mrca"].astype(float).round(3)

    enh["enh_len"]= enh["end_enh"]- enh["start_enh"] # enhancer lengths
    enh["syn_len"]= enh["end_syn"]- enh["start_syn"] # syntenic lengths
    enh["syn_id"] = enh["chr_syn"]+ ":" + enh["start_syn"].map(str) + "-" + enh["end_syn"].map(str)

    ########
    # calculate TFBS density
    #
    # count tfs associated with enhancers
    # count uniq tfs associated with enhancers
    # count syn tfs associated with syntenic blocks
    # count uniq syn tfs associated with syntenic blocks
    ########

    enh["tf_bin"] = 0
    enh["tf_bin"].loc[enh.len_tfbs_overlap >5] = 1

    # syn_tf_den

    syntf_count = enh.groupby(['enh_id','syn_id', 'mrca', 'seg_index', 
                               'core', 'core_remodeling', 'syn_len'])["tf_bin"].sum().reset_index()

    syntf_count["syn_tf_den"] = syntf_count.tf_bin.divide(syntf_count.syn_len) # calculate syn_density

    syntf_count.columns = ['enh_id','syn_id', 'mrca', 'seg_index', 
                               'core', 'core_remodeling', 'syn_len', 'syn_tf_count', 'syn_tf_den'] # rename columns

    # syn_tf_u_den

    syntfu_count = enh[['enh_id','syn_id', 'seg_index', 'enh_len', 'syn_len', 'tf', 'tf_bin']]\
    .drop_duplicates().reset_index() # get unique tf density

    syntfu_count = syntfu_count.groupby(['enh_id','syn_id', 'seg_index', 'enh_len', 'syn_len'])["tf_bin"]\
    .sum().reset_index()

    syntfu_count["syn_tf_u_den"] = syntfu_count.tf_bin.divide(syntfu_count.syn_len) # calculate syn_density

    syntfu_count.columns = ['enh_id','syn_id', 'seg_index', 'enh_len', 'syn_len', 'syn_tf_u_count', 'syn_tf_u_den'] # rename columns

    # enh_tf_den

    enhtf_count = enh.groupby(['enh_id','core_remodeling', 'enh_len'])["tf_bin"].sum().reset_index()

    enhtf_count["enh_tf_den"] = enhtf_count.tf_bin.divide(enhtf_count.enh_len) # calculate syn_density

    enhtf_count.columns = ['enh_id','core_remodeling', 'enh_len', 'enh_tf_count', 'enh_tf_den'] # rename columns

    enh_mrca = enh.groupby(['enh_id'])['mrca'].max().reset_index()

    enhtf_count = pd.merge(enhtf_count, enh_mrca, how = "left", on = "enh_id") # get max age

    # enh_tf_u_den

    enhtfu_count = enh[['enh_id','enh_len', 'tf', "tf_bin"]].drop_duplicates().reset_index() # get unique tf density

    enhtfu_count = enhtfu_count.groupby(['enh_id','enh_len'])["tf_bin"].sum().reset_index()

    enhtfu_count["enh_tf_u_den"] = enhtfu_count.tf_bin.divide(enhtfu_count.enh_len) # calculate syn_density

    enhtfu_count.columns = ['enh_id', 'enh_len', 'enh_tf_u_count', 'enh_tf_u_den'] # rename columns

    enh = pd.merge(enhtf_count, enhtfu_count, how = "left", on = (['enh_id','enh_len']))

    syn = pd.merge(syntf_count, syntfu_count, how = "left", on =(['enh_id','syn_id', 'seg_index', 'syn_len']))

    with open(syn_out, 'a') as f:
        syn.to_csv(f, sep = '\t', header = True, index = False) # write last enhancers  

    with open(enh_out, 'a') as f:
        enh.to_csv(f, sep = '\t', header = True, index = False) # write last enhancers  

    print("Finished calculating SYN TFBS density", datetime.now())  

/dors/capra_lab/projects/enhancer_ages/fantom/data/download/tfbs/shuf-all_fantom_enh_112_tissues-54_age_breaks_syn_tfbs_density.bed
2654 raw df len
the number syntenic blocks that did not overlap TFBS 1797
the number of unique enhancers 1200


/dors/capra_lab/users/fongsl/sfenv/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Finished calculating SYN TFBS density 2020-03-11 14:09:44.464857
/dors/capra_lab/projects/enhancer_ages/fantom/data/download/tfbs/shuf-all_fantom_enh_112_tissues-64_age_breaks_syn_tfbs_density.bed
92108 raw df len
the number syntenic blocks that did not overlap TFBS 66507
the number of unique enhancers 42235


/dors/capra_lab/users/fongsl/sfenv/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Finished calculating SYN TFBS density 2020-03-11 14:10:54.379251
/dors/capra_lab/projects/enhancer_ages/fantom/data/download/tfbs/shuf-all_fantom_enh_112_tissues-49_age_breaks_syn_tfbs_density.bed
65713 raw df len
the number syntenic blocks that did not overlap TFBS 47748
the number of unique enhancers 30472


...

Finished calculating SYN TFBS density 2020-03-11 15:50:10.165018
/dors/capra_lab/projects/enhancer_ages/fantom/data/download/tfbs/shuf-all_fantom_enh_112_tissues-57_age_breaks_syn_tfbs_density.bed
79622 raw df len
the number syntenic blocks that did not overlap TFBS 56830
the number of unique enhancers 36396


/dors/capra_lab/users/fongsl/sfenv/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Finished calculating SYN TFBS density 2020-03-11 15:51:02.207285
/dors/capra_lab/projects/enhancer_ages/fantom/data/download/tfbs/shuf-all_fantom_enh_112_tissues-81_age_breaks_syn_tfbs_density.bed
66679 raw df len
the number syntenic blocks that did not overlap TFBS 47830
the number of unique enhancers 30470


/dors/capra_lab/users/fongsl/sfenv/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Finished calculating SYN TFBS density 2020-03-11 15:52:02.463912


In [48]:
syn_out

'/dors/capra_lab/projects/enhancer_ages/fantom/data/download/tfbs/CL0000047_syn_tfbs_density.bed'

In [54]:
enh.head()

,enh_id,core_remodeling,enh_len,enh_tf_count,enh_tf_den,mrca,enh_tf_u_count,enh_tf_u_den
0,chr10:101475394-101475766,1,372,2,0.005376,0.175,2,0.005376
1,chr10:104545681-104545915,0,234,1,0.004274,0.175,1,0.004274
2,chr10:104556220-104556459,1,239,20,0.083682,0.308,19,0.079498
3,chr10:105302775-105302952,1,177,9,0.050847,0.380,9,0.050847
4,chr10:105307461-105307595,0,134,2,0.014925,0.175,2,0.014925


In [50]:
syn.head()

,enh_id,syn_id,mrca,seg_index,core,core_remodeling,syn_len,syn_tf_count,syn_tf_den,enh_len,syn_tf_u_count,syn_tf_u_den
0,chr10:101475394-101475766,chr10:101475394-101475765,0.175,0,1,1,371,2,0.005391,372,2,0.005391
1,chr10:101475394-101475766,chr10:101475765-101475766,0.152,1,0,1,1,0,0.000000,372,0,0.000000
2,chr10:104545681-104545915,chr10:104545681-104545915,0.175,0,1,0,234,1,0.004274,234,1,0.004274
3,chr10:104556220-104556459,chr10:104556220-104556413,0.308,0,1,1,193,18,0.093264,239,18,0.093264
4,chr10:104556220-104556459,chr10:104556413-104556459,0.175,1,0,1,46,2,0.043478,239,2,0.043478
